# Importações

In [ ]:
import cv2
import numpy as np
from skimage.draw import line_aa

# Visualização do HOG em tempo real

## Visualização padrão skimage

In [ ]:
cell_size = 8
magnitude_threshold = 10  
n_bins = 9

def get_hog_histogram(magnitude, angle_rad, cell_size, n_bins):
  
    H, W = magnitude.shape
    n_cells_y = H // cell_size
    n_cells_x = W // cell_size

    hog_cells = np.zeros((n_cells_y, n_cells_x, n_bins), dtype=float)
    angle_unit = np.pi / n_bins  

    for y in range(n_cells_y):
        for x in range(n_cells_x):
            y_start, x_start = y * cell_size, x * cell_size
            cell_mag = magnitude[y_start : y_start + cell_size, x_start : x_start + cell_size]
            cell_ang = angle_rad[y_start : y_start + cell_size, x_start : x_start + cell_size]

            bin_indices = np.floor(cell_ang / angle_unit).astype(int)

            bin_indices = np.clip(bin_indices, 0, n_bins - 1)

            bincount = np.bincount(bin_indices.ravel(), weights=cell_mag.ravel(), minlength=n_bins)
            hog_cells[y, x, :] = bincount

    return hog_cells

def view_hog_overlay(frame, hog_cells, cell_size):
    H, W = frame.shape[:2]
    n_cells_y, n_cells_x, n_bins = hog_cells.shape

    hog_layer = np.zeros((H, W), dtype=float)

    bin_centers = (np.arange(n_bins) + 0.5) * (np.pi / n_bins)
    radius = int(cell_size / 2)

    max_val = hog_cells.max()
    if max_val > 0:
        cells_norm = hog_cells.astype(float) / max_val
    else:
        cells_norm = hog_cells.astype(float)

    for i in range(n_cells_y):
        for j in range(n_cells_x):
            cx = j * cell_size + cell_size // 2
            cy = i * cell_size + cell_size // 2

            for b in range(n_bins):
                strength = cells_norm[i, j, b]
                if strength < 0.05: 
                    continue

                angle = bin_centers[b] + (np.pi / 2.0)

                angle = np.mod(angle, np.pi)

                dx = radius * np.cos(angle)
                dy = radius * np.sin(angle)

                r0 = int(round(cy - dy))
                c0 = int(round(cx - dx))
                r1 = int(round(cy + dy))
                c1 = int(round(cx + dx))

                rr, cc, val = line_aa(r0, c0, r1, c1)
                valid = (rr >= 0) & (rr < H) & (cc >= 0) & (cc < W)
                if np.any(valid):
                    hog_layer[rr[valid], cc[valid]] += val[valid] * strength

    if hog_layer.max() > 0:
        hog_layer = hog_layer / hog_layer.max()

    hog_layer = (hog_layer * 255).astype(np.uint8)

    hog_layer_color = cv2.merge([hog_layer, hog_layer, hog_layer])

    return hog_layer_color

In [6]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Erro: Não foi possível acessar a webcam.")
print("Pressione 'q' para sair")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_invertido = cv2.flip(frame, 1)
    frame = cv2.resize(frame_invertido, (1280, 720))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    magnitude = cv2.magnitude(gx, gy)
    angle_rad = cv2.phase(gx, gy, angleInDegrees=False)
    angle_rad = np.mod(angle_rad, np.pi)

    hog_cells = get_hog_histogram(magnitude, angle_rad, cell_size, n_bins)
    frame_com_hog = view_hog_overlay(frame, hog_cells, cell_size)

    cv2.imshow('HOG Real-Time Overlay', frame_com_hog)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Visualização com setas

In [ ]:
cap = cv2.VideoCapture(0) 

cell_size = 8 
magnitude_threshold = 10
scale_factor = 0.1

cv2.namedWindow("HOG - Visualização em tempo real (Corrigida)", cv2.WINDOW_NORMAL)

if not cap.isOpened():
    print("Erro: Não foi possível acessar a webcam.")

print("Pressione 'q' para sair")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_invertido = cv2.flip(frame, 1)
    frame = cv2.resize(frame_invertido, (1280, 720))
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    H, W = gray.shape

    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    magnitude = cv2.magnitude(gx, gy)

    for y in range(0, H, cell_size):
        for x in range(0, W, cell_size):
            
            cell_mag = magnitude[y:y+cell_size, x:x+cell_size]
            cell_gx  = gx[y:y+cell_size, x:x+cell_size]
            cell_gy  = gy[y:y+cell_size, x:x+cell_size]

            avg_mag = np.mean(cell_mag)
            
            if avg_mag > magnitude_threshold:
                
                avg_gx = np.mean(cell_gx)
                avg_gy = np.mean(cell_gy)
                
                avg_ang = np.arctan2(avg_gy, avg_gx)

                center_x = x + cell_size // 2
                center_y = y + cell_size // 2

                arrow_length = avg_mag * scale_factor
                
                if arrow_length > cell_size * 2:
                    arrow_length = cell_size * 2

                end_x = int(center_x + arrow_length * np.cos(avg_ang))
                end_y = int(center_y + arrow_length * np.sin(avg_ang))

                cv2.arrowedLine(frame, (center_x, center_y), (end_x, end_y),
                                (0, 0, 255), 1, tipLength=0.3)

    cv2.imshow("HOG - Visualização em tempo real (Corrigida)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()